This notebook demonstrates a basic SQL agent that translates natural language questions into SQL queries.

In [1]:
import json
import os
from typing import Annotated, Dict

from autogen import ConversableAgent, UserProxyAgent, AssistantAgent, config_list_from_json
from dotenv import load_dotenv
import autogen
from typing import Annotated
from openai import AzureOpenAI
import sqlite3
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.utilities.sql_database import SQLDatabase
from langchain.llms import OpenAI

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_GPT4o_DEPLOYMENT=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_KEY=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")


In [2]:
llm = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_KEY,
        api_version=AZURE_OPENAI_API_VERSION
)

## Agent Implementation

Using AutoGen, a SQL agent can be implemented with an agent. It executes the generated SQL query and the agent can take execution results as feedback to improve its generation in multiple rounds of conversations.

In [3]:
llm_config = {
    "cache_seed": 45,  # change the cache_seed for different trials
    "temperature": 0,
    "timeout": 120,  # in seconds
    "config_list": 
    [
        {
            "model": AZURE_OPENAI_GPT4o_DEPLOYMENT,
            "api_type": "azure",
            "api_key": AZURE_OPENAI_KEY,
            "base_url": AZURE_OPENAI_ENDPOINT,
            "api_version": AZURE_OPENAI_API_VERSION
        }
    ]
}

## Architecture  
### Architecture 1: Few Shot SQL Generation 
![Architecture 2: Few Shot SQL Generation](./assets/Few%20Shot%20SQL%20generation.png)  

If your schema is more complex, you can use a Vector DB to store the tables definitions (the metadata) and use a RAG agent 
to get the relevant table information to send as a schema.
### Architecture 2: Few Shot SQL Generation with RAG
![Architecture 2: Few Shot SQL Generation with RAG](./assets/Few%20Shot%20SQL%20Generation%20with%20RAG.png)

Sources:
[NL to SQL Architecture Alternatives](https://techcommunity.microsoft.com/blog/azurearchitectureblog/nl-to-sql-architecture-alternatives/4136387)

In [4]:
schema= """
    CREATE TABLE IF NOT EXISTS Authors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            biography TEXT
        );

    CREATE TABLE IF NOT EXISTS Publishers (
        d INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        address TEXT
    );      

    CREATE TABLE IF NOT EXISTS Books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        price REAL,
        author_id INTEGER NOT NULL, FOREIGN KEY (author_id) REFERENCES Authors(id)
    ); 
"""

In [22]:
def check_termination(msg: Dict):
    if "tool_responses" not in msg:
        return False
    json_str = msg["tool_responses"][0]["content"]
    obj = json.loads(json_str)
    return "error" not in obj or obj["error"] is None and obj["reward"] == 1

general_assistant = autogen.AssistantAgent(
    name="general_assistant",
    description="Call this agent to provide answers not related to the Bookstore SQL database.",
    system_message="""
    You are a helpful assistant. You must always reply with some form of text.
    All questions not related to the Bookstore SQL database should be answered with the response :
    "I am an expert at providing facts from an SQL Database. Please ask me a question related to SQL only"
    Answer with TERMINATE if you want to end the conversation.
    """,
    llm_config=llm_config,
    is_termination_msg=check_termination,
    human_input_mode="NEVER",
)

# using the Few-shot approach to train the model
db_assistant = autogen.AssistantAgent(
    name="db_assistant",
    description="Call this agent to provide answers using Natural Language based on the SQL answers provided by the sqlwriter agent.",
    system_message="""
    You are a helpful assistant. You must always reply with some form of text.
   
    If the question is related to the Bookstore SQL database, answer the question by turning the SQL result you are given to 
    a truthful and accurate natural language response to the original question.
    For example:

    User Question: How many books are there in the bookstore?
    Answer: There are 10 books in the bookstore.

    User Question: How many books by Herman Melville are in the bookstore?
    Answer: There are 0 books in the bookstore written by Herman Melville.

    User Question: How many publishers for the book Moby Dick?
    Answer: There is no information about publishers in the bookstore.

    User Question: What is the weather?
    Answer: I am an expert at providing facts from a SQL Database. Please ask me a question related to SQL only.
    
    """,
    llm_config=llm_config,
    is_termination_msg=check_termination,
    human_input_mode="NEVER",
)



In [23]:
sql_writer = AssistantAgent(
    name="sql_writer",
    description="Call this agent when the question is related to the Bookstore DB.",
    llm_config=llm_config,
    system_message="""
    You are good at writing SQL queries for a bookstore DB.
    Use the DB schema below to answer the questions:
    {schema}
    Always respond with a function call to execute_sql().
    """,
    is_termination_msg=check_termination,
    human_input_mode="NEVER",
)

user_proxy = UserProxyAgent(
    "user_proxy", 
    human_input_mode="NEVER", 
    max_consecutive_auto_reply=5, 
    code_execution_config={
        "work_dir":"coding", 
        "use_docker":False
    }
)

@sql_writer.register_for_llm(description="Function for executing SQL query and returning a response")
@user_proxy.register_for_execution()
def execute_sql(sql: Annotated[str, "SQL query"]) -> Annotated[str, "results"]:
    database = 'bookstore.db'

    conn = sqlite3.connect(database)
    cur = conn.cursor()

    try:
        cur.execute(sql)
        rows = cur.fetchall()
        return rows
    except Exception as e:
        return {
            "error": "The SQL query returned an incorrect result",
            "wrong_result": str(e)
        }
    finally:
        cur.close()
        conn.close()


In [24]:
group_chat = autogen.GroupChat(
    agents=[user_proxy, sql_writer, general_assistant, db_assistant], messages=[], max_round=5
)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

In [25]:
# send the autogen logs to a local db (sqlite) 
# you can download the sqlitebrowser and check the prompts being sent to the LLM
logging_session_id = autogen.runtime_logging.start(config={"dbname": "logs.db"})
print("Logging session ID: " + str(logging_session_id))

INFO:autogen.logger.sqlite_logger:no migration scripts, skip...


Logging session ID: 27dcd837-707d-42e9-b031-7571806eb90b


In [27]:
question = "How many books are in the bookstore?"

user_proxy.initiate_chat(
    manager,
    message=question,
)

user_proxy (to chat_manager):

How many books are in the bookstore?

--------------------------------------------------------------------------------



Next speaker: sql_writer



sql_writer (to chat_manager):

***** Suggested tool call (call_6JZYyajxqBaMun0Ws8Zc70NI): execute_sql *****
Arguments: 
{"sql":"SELECT COUNT(*) FROM books;"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION execute_sql...
user_proxy (to chat_manager):

***** Response from calling tool (call_6JZYyajxqBaMun0Ws8Zc70NI) *****
[[2]]
**********************************************************************

--------------------------------------------------------------------------------



Next speaker: db_assistant



ChatResult(chat_id=None, chat_history=[{'content': 'How many books are in the bookstore?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_6JZYyajxqBaMun0Ws8Zc70NI', 'function': {'arguments': '{"sql":"SELECT COUNT(*) FROM books;"}', 'name': 'execute_sql'}, 'type': 'function'}], 'name': 'sql_writer', 'role': 'assistant'}, {'content': '[[2]]', 'tool_responses': [{'tool_call_id': 'call_6JZYyajxqBaMun0Ws8Zc70NI', 'role': 'tool', 'content': '[[2]]'}], 'role': 'tool', 'name': 'user_proxy'}], summary='[[2]]', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [28]:
question = "what is the weather?"
user_proxy.initiate_chat(
    manager,
    message=question,
)

user_proxy (to chat_manager):

what is the weather?

--------------------------------------------------------------------------------



Next speaker: general_assistant



general_assistant (to chat_manager):

I am an expert at providing facts from an SQL Database. Please ask me a question related to SQL only.

--------------------------------------------------------------------------------



Next speaker: general_assistant



general_assistant (to chat_manager):

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'what is the weather?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'I am an expert at providing facts from an SQL Database. Please ask me a question related to SQL only.', 'name': 'general_assistant', 'role': 'user'}, {'content': 'TERMINATE', 'name': 'general_assistant', 'role': 'user'}], summary='', cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=[])

In [29]:
question = "How many books by Harper Lee are in the bookstore?"
user_proxy.initiate_chat(
    manager,
    message=question,
)

user_proxy (to chat_manager):

How many books by Harper Lee are in the bookstore?

--------------------------------------------------------------------------------



Next speaker: sql_writer



sql_writer (to chat_manager):

***** Suggested tool call (call_PG7xFTYNsllGDhIbfRPfCKur): execute_sql *****
Arguments: 
{"sql":"SELECT COUNT(*) FROM books WHERE author = 'Harper Lee';"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION execute_sql...
user_proxy (to chat_manager):

***** Response from calling tool (call_PG7xFTYNsllGDhIbfRPfCKur) *****
{"error": "The SQL query returned an incorrect result", "wrong_result": "no such column: author"}
**********************************************************************

--------------------------------------------------------------------------------



Next speaker: sql_writer



sql_writer (to chat_manager):

***** Suggested tool call (call_VNF80vc2cHMan8PidVYt9gJx): execute_sql *****
Arguments: 
{"sql":"SELECT COUNT(*) FROM books WHERE author_id = (SELECT id FROM authors WHERE name = 'Harper Lee');"}
****************************************************************************

--------------------------------------------------------------------------------

Next speaker: user_proxy


>>>>>>>> EXECUTING FUNCTION execute_sql...
user_proxy (to chat_manager):

***** Response from calling tool (call_VNF80vc2cHMan8PidVYt9gJx) *****
[[1]]
**********************************************************************

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'How many books by Harper Lee are in the bookstore?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_PG7xFTYNsllGDhIbfRPfCKur', 'function': {'arguments': '{"sql":"SELECT COUNT(*) FROM books WHERE author = \'Harper Lee\';"}', 'name': 'execute_sql'}, 'type': 'function'}], 'name': 'sql_writer', 'role': 'assistant'}, {'content': '{"error": "The SQL query returned an incorrect result", "wrong_result": "no such column: author"}', 'tool_responses': [{'tool_call_id': 'call_PG7xFTYNsllGDhIbfRPfCKur', 'role': 'tool', 'content': '{"error": "The SQL query returned an incorrect result", "wrong_result": "no such column: author"}'}], 'role': 'tool', 'name': 'user_proxy'}, {'content': 'None', 'tool_calls': [{'id': 'call_VNF80vc2cHMan8PidVYt9gJx', 'function': {'arguments': '{"sql":"SELECT COUNT(*) FROM books WHERE author_id = (SELECT id FROM authors WHERE name = \'Harper Lee\');"}', 'name': 'execute_sql'}, '

In [30]:
autogen.runtime_logging.stop()